In [11]:
import fitsio as fio
import numpy as np
import os, sys
import glob
from matplotlib import pyplot as plt
import proplot as pplt
from tqdm import tqdm
import pickle
from des_y6utils import mdet

os.environ["MEDS_DIR"] = "/global/cfs/cdirs/des/y6-shear-catalogs"

# Accessing metadetection files on NERSC. 

In [2]:
# grab mdet files (these files are divided into patches)
mdet_files = glob.glob('/global/cfs/cdirs/des/y6-shear-catalogs/metadetection_patches_v1_blinded/patch-*.fits')

In [3]:
print(len(mdet_files))
d = fio.read(mdet_files[0])
print(d.dtype)

200
[('uid', '>i8'), ('patch_num', '>i2'), ('tilename', '<U12'), ('slice_id', '>i2'), ('mdet_step', '<U7'), ('ra', '>f8'), ('dec', '>f8'), ('x', '>f4'), ('y', '>f4'), ('mfrac', '>f4'), ('mfrac_img', '>f4'), ('nepoch_g', '>i4'), ('nepoch_r', '>i4'), ('nepoch_i', '>i4'), ('nepoch_z', '>i4'), ('psfrec_g_1', '>f8'), ('psfrec_g_2', '>f8'), ('psfrec_T', '>f4'), ('wmom_s2n', '>f4'), ('wmom_g_1', '>f8'), ('wmom_g_2', '>f8'), ('wmom_g_cov_1_1', '>f4'), ('wmom_g_cov_1_2', '>f4'), ('wmom_g_cov_2_2', '>f4'), ('wmom_T_err', '>f4'), ('wmom_T_ratio', '>f4'), ('wmom_psf_T', '>f4'), ('pgauss_T_err', '>f4'), ('pgauss_T', '>f4'), ('pgauss_psf_T', '>f4'), ('pgauss_band_flux_g', '>f4'), ('pgauss_band_flux_r', '>f4'), ('pgauss_band_flux_i', '>f4'), ('pgauss_band_flux_z', '>f4'), ('pgauss_band_flux_err_g', '>f4'), ('pgauss_band_flux_err_r', '>f4'), ('pgauss_band_flux_err_i', '>f4'), ('pgauss_band_flux_err_z', '>f4')]


# Computing shear response over all the catalogs. 
### (* no response for each object for metadetection!)

In [4]:
def _accum_shear_per_tile(res, mdet_step, g1, g2):

    """
    Returns the dictionary of the accumulated shear (sum of individual shear).

    Parameters
    ----------
    res: A dictionary in which accumulated sums of shear are stored
    mdet_step: An array of metadetection steps (noshear, 1p, 1m, 2p, 2m) for each object in metadetection catalog
    g1: An array of the measured shapes (e1) for each object in metadetection catalog
    g2: An array of the measured shapes (e2) for each object in metadetection catalog

    """
    for step in ['noshear', '1p', '1m', '2p', '2m']:
        msk_s = np.where(mdet_step == step)[0]
        
        np.add.at(
            res[step], 
            (0, 0), 
            np.sum(g1[msk_s]),
        )
        np.add.at(
            res[step], 
            (0, 1), 
            np.sum(g2[msk_s]),
        )
        np.add.at(
            res["num_" + step], 
            (0, 0), 
            len(g1[msk_s]),
        )
        np.add.at(
            res["num_" + step], 
            (0, 1), 
            len(g2[msk_s]),
        )
    return res

In [7]:
binnum = 1
res = {'noshear': np.zeros((binnum, 2)), 'num_noshear': np.zeros((binnum, 2)), 
       '1p': np.zeros((binnum, 2)), 'num_1p': np.zeros((binnum, 2)), 
       '1m': np.zeros((binnum, 2)), 'num_1m': np.zeros((binnum, 2)),
       '2p': np.zeros((binnum, 2)), 'num_2p': np.zeros((binnum, 2)),
       '2m': np.zeros((binnum, 2)), 'num_2m': np.zeros((binnum, 2))}

for fname in tqdm(mdet_files):
    
    try:
        d = fio.read(fname)
    except:
        print('this file cannot be read', fname)
        continue
    
    msk = mdet.make_mdet_cuts(d, 1) # the second argument is ID of the version of the cuts. the most recent ID is 2. 
    d = d[msk]
    
    res = _accum_shear_per_tile(res, d['mdet_step'], d['wmom_g_1'], d['wmom_g_2'])

100%|██████████| 200/200 [1:10:19<00:00, 21.10s/it]


In [8]:
# computing shear response over all the catalogs.
g1 = res['noshear'][0][0] / res['num_noshear'][0][0]
g1p = res['1p'][0][0] / res['num_1p'][0][0]
g1m = res['1m'][0][0] / res['num_1m'][0][0]
R11 = (g1p - g1m) / 2 / 0.01

g2 = res['noshear'][0][1] / res['num_noshear'][0][1]
g2p = res['2p'][0][1] / res['num_2p'][0][1]
g2m = res['2m'][0][1] / res['num_2m'][0][1]
R22 = (g2p - g2m) / 2 / 0.01

R = (R11+R22)/2
print(R11, R22, R)

# off-diagonal part
g2p_g1 = res['2p'][0][0] / res['num_2p'][0][0]
g2m_g1 = res['2m'][0][0] / res['num_2m'][0][0]
g1p_g2 = res['1p'][0][1] / res['num_1p'][0][1]
g1m_g2 = res['1m'][0][1] / res['num_1m'][0][1]
R12 = (g2p_g1 - g2m_g1)/2/0.01
R21 = (g1p_g2 - g1m_g2)/2/0.01
print(R12, R21, (R12+R21)/2)

0.2813003824884869 0.2819703494270017 0.2816353659577443
4.4005294846831205e-06 7.919873190585677e-05 4.1799630695269946e-05


# Using shear to do various things!

## For things that do not care about jackknife patches (e.g., getting shear weight). 

In [13]:
# preliminary shear weight computed in the bins of S/N and size ratio. Similar to Fig. 4 in Y3 catalog paper.
# If you're using the cut ID = 2, you'd want to use inverse_variance_weight_final_v2.pickle, rather than v1. 
with open('/global/homes/m/myamamot/DES/des-y6-analysis/y6_measurement/inverse_variance_weight_final_v1.pickle', 'rb') as handle:
    wgt_dict = pickle.load(handle)

In [15]:
def assign_loggrid(x, y, xmin, xmax, xsteps, ymin, ymax, ysteps):
    from math import log10
    # return x and y indices of data (x,y) on a log-spaced grid that runs from [xy]min to [xy]max in [xy]steps

    logstepx = log10(xmax/xmin)/xsteps
    logstepy = log10(ymax/ymin)/ysteps

    indexx = (np.log10(x/xmin)/logstepx).astype(int)
    indexy = (np.log10(y/ymin)/logstepy).astype(int)

    indexx = np.maximum(indexx,0)
    indexx = np.minimum(indexx, xsteps-1)
    indexy = np.maximum(indexy,0)
    indexy = np.minimum(indexy, ysteps-1)

    return indexx,indexy

def _find_shear_weight(d, wgt_dict, snmin, snmax, sizemin, sizemax, steps):
    
    if wgt_dict is None:
        weights = np.ones(len(d))
        return weights

    shear_wgt = wgt_dict['weight']
    shear_response = wgt_dict['response']
    indexx, indexy = assign_loggrid(d['wmom_s2n'], d['wmom_T_ratio'], snmin, snmax, steps, sizemin, sizemax, steps)
    weights = np.array([shear_wgt[x, y] for x, y in zip(indexx, indexy)])
    response = np.array([shear_response[x, y] for x, y in zip(indexx, indexy)])
    
    return response, weights

In [ ]:
# This is where all the objects after selections are concatenated in an array.
obj_list = []
for fname in tqdm(mdet_files):
    
    try:
        # Read in a file. 
        d = fio.read(fname)
        # Call the external function from https://github.com/des-science/des-y6utils to apply selection cuts.
        # Please check the github page for further details on what kinds of cuts are applied. 
        # The second argument is ID of the version of the cuts. the most recent ID is 2. 
        msk = mdet.make_mdet_cuts(d, 1) 
        # Also select objects in unsheared images.
        noshear_mask = (d['mdet_step'] == 'noshear')
        d = d[msk & noshear_mask]
    except:
        continue
    
    # If you'd need less or more columns (e.g., shape variance), you'd need to remove or add them here. 
    mdet_obj = np.zeros(len(d), dtype=[('ra', 'f8'), ('dec', 'f8'), ('s2n', 'f8'), ('g1', 'f8'), ('g2', 'f8'), ('R_all', 'f8'), ('R_w', 'f8'), ('w', 'f8'), ('band_flux_g', 'f8'), ('band_flux_r', 'f8'), ('band_flux_i', 'f8'), ('band_flux_z', 'f8'), ('band_flux_err_g', 'f8'), ('band_flux_err_r', 'f8'), ('band_flux_err_i', 'f8'), ('band_flux_err_z', 'f8')])
    # Based on each object's S/N and size ratio, shear weight is decided. 
    shear_response, shear_wgt = _find_shear_weight(d, wgt_dict, 10, 500, 1.2, 2.0, 20)
    
    mdet_obj['ra'] = d['ra']
    mdet_obj['dec'] = d['dec']
    mdet_obj['s2n'] = d['wmom_s2n']
    mdet_obj['g1'] = d['wmom_g_1'] # raw shear (uncorrected for the response)
    mdet_obj['g2'] = d['wmom_g_2'] # raw shear (uncorrected for the response)
    mdet_obj['R_all'] = R # global shear response computed in previous cells in this notebook.
    
    mdet_obj['R_w'] = shear_response # shear response computed in the bins of S/N and size ratio.
    mdet_obj['w'] = shear_wgt # shear weight computed in the bins of S/N and size ratio. 
    
    mdet_obj['band_flux_g'] = d['pgauss_band_flux_g']
    mdet_obj['band_flux_r'] = d['pgauss_band_flux_r']
    mdet_obj['band_flux_i'] = d['pgauss_band_flux_i']
    mdet_obj['band_flux_z'] = d['pgauss_band_flux_z']
    mdet_obj['band_flux_err_g'] = d['pgauss_band_flux_err_g']
    mdet_obj['band_flux_err_r'] = d['pgauss_band_flux_err_r']
    mdet_obj['band_flux_err_i'] = d['pgauss_band_flux_err_i']
    mdet_obj['band_flux_err_z'] = d['pgauss_band_flux_err_z']
    
    obj_list.append(mdet_obj)

mdet_all = np.concatenate(obj_list)
# save the array if you'd like, although this might cause memory issue. 
# fio.write('metadetection_v1.fits', mdet_all)

 34%|███▍      | 68/200 [26:30<49:17, 22.40s/it]  

## For things that do care about jackknife patches (e.g., mean shear vs PSF shape). 

In [ ]:
# Your own analysis code with jackknife errors. 